In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.tools.render import format_tool_to_openai_function
from langchain.prompts import ChatPromptTemplate
from utils import get_openai_api_key
from Tools.temperature import get_current_temperature
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.agent import AgentFinish
import openai

openai.api_key = get_openai_api_key()
functions = [
        format_tool_to_openai_function(f) for f in [get_current_temperature]
]
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant. If user ask you about movie, you should answer sincerely. \
        if you don't have information of user, suppose that the user is in Seoul and don't ask user"),
    ("user", "{input}"),
])

def route(result):
	if isinstance(result, AgentFinish): # 함수를 쓰지 않기로 결정한다면 -> content 반환
		return result.return_values['output']
	else: # 함수를 쓰기로 결정한다면 -> 함수명에 따라 어떤 함수를 쓸지 결정해주고, argument를 넣은 값 반환
		tools = {
			"get_current_temperature": get_current_temperature,
		}
		return tools[result.tool].run(result.tool_input)
  
model = ChatOpenAI(temperature=0).bind(functions=functions)
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route
chain.invoke({"input":"오늘 부산 날씨는 어떤것 같아"})

'The current temperature is -3.5°C'

In [3]:
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a movie recommender. Use your knowledge to recommend movies that fit\
        and rewrite with given informations {informations}. You shoud answer using only Korean and markdown structure"),
])

In [4]:
model2 = ChatOpenAI(temperature=0, max_tokens=1024)
chain2 = prompt2 | model2

In [7]:
result1 = informations=chain.invoke({"input":"오늘 날씨에 맞는 영화 추천좀"})
result2 = chain2.invoke({"informations": result1})

In [9]:
print(result2.content)

현재 온도는 -8.2°C입니다. 추운 날씨에는 온기를 느낄 수 있는 영화를 추천해드릴게요. 아래의 영화들을 즐기시면 추위를 잊고 영화 속에서 따뜻한 분위기를 느낄 수 있을 거예요:

1. 겨울왕국 (Frozen): 겨울 풍경과 따뜻한 가족 사랑을 다룬 애니메이션 영화입니다. 눈과 얼음으로 뒤덮인 왕국에서 펼쳐지는 이야기는 추운 날씨에도 마음을 녹일 수 있어요.

2. 러브 액츄얼리 (Love Actually): 크리스마스 분위기와 사랑 이야기가 어우러진 영화입니다. 다양한 인물들의 이야기가 따뜻한 감동을 전해줄 거예요.

3. 캐롤 (Carol): 1950년대 겨울을 배경으로 한 로맨스 드라마입니다. 눈이 내리는 도시에서 펼쳐지는 사랑 이야기는 추운 날씨에 어울리는 영화 중 하나에요.

4. 홈 애로인 (Home Alone): 겨울 방학 동안 집에 혼자 남겨진 소년의 모험을 그린 코미디 영화입니다. 따뜻한 가족의 사랑과 유쾌한 이야기로 추위를 잊게 해줄 거예요.

이 영화들을 시청하면 추운 날씨에도 따뜻한 감정을 느낄 수 있을 거예요. 즐겁게 영화를 감상하세요!
